In [16]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [8]:
url = 'https://rbbv.com.br/destino/america-do-sul/'
f = urlopen(url)

In [20]:
def get_content(url):
    f = urlopen(url)
    return BeautifulSoup(f)

In [9]:
soup = BeautifulSoup(f)

In [9]:
see_all = soup.find_all('h3', { "class": "see-all" })

In [12]:
all_h3 = soup.find_all('h3')

In [20]:
[h3.a.get('href') for h3 in all_h3 if h3.a]

['https://rbbv.com.br/destino/america-do-sul/argentina/',
 'https://rbbv.com.br/destino/america-do-sul/bolivia/',
 'https://rbbv.com.br/destino/america-do-sul/brasil/',
 'https://rbbv.com.br/destino/america-do-sul/chile/',
 'https://rbbv.com.br/destino/america-do-sul/colombia/',
 'https://rbbv.com.br/destino/america-do-sul/equador/',
 'https://rbbv.com.br/destino/america-do-sul/paraguai/',
 'https://rbbv.com.br/destino/america-do-sul/peru/',
 'https://rbbv.com.br/destino/america-do-sul/uruguai/',
 'https://rbbv.com.br/destino/america-do-sul/venezuela/']

In [1]:
def obter_links_grupo_maior(grupo):
    link_grupo = f'https://rbbv.com.br/destino/{grupo}/'
    f = urlopen(link_grupo)
    soup = BeautifulSoup(f)
    all_h3 = soup.find_all('h3')
    links = [(h3.text.replace('Ver Todos', ''), h3.a.get('href')) for h3 in all_h3 if h3.a]
    return pd.DataFrame(links, columns=['pais', 'link'])

In [5]:
an = obter_links_grupo_maior('america-do-norte')

In [6]:
an

['https://rbbv.com.br/destino/america-do-norte/canada/',
 'https://rbbv.com.br/destino/america-do-norte/eua/',
 'https://rbbv.com.br/destino/america-do-norte/mexico/']

In [11]:
grupos_maiores = ['america-do-sul', 'america-do-norte', 'america-central', 'africa']

## Desafios

- Tipos páginas no menu
  * Links para agrupadores de Links
    * Ideia: links começam com 'https://rbbv.com'
  * Links para posts  
    * Ideia: links não começam com 'https://rbbv.com'

### Tentativa: obter lista de agrupadores de links pelo Menu Destinos

In [22]:
c = get_content('https://rbbv.com.br/destino/america-do-sul/')
menu_destinos = c.find('ul', { 'id': 'menu-destinos' })

In [24]:
# Nao faz diferenca se colocar a property recursive=True
all_li = menu_destinos.find_all('li')
len(all_li)

In [57]:
l = [(li.a.text, li.a.get('href')) for li in all_li if li.a]

In [74]:
df_destinos = pd.DataFrame(l, columns=['Destino', 'Link'])
df_destinos = df_destinos.dropna()
df_destinos['Referencias'] = count_references_all(l)

In [78]:
print(df_destinos.shape)
df_destinos.head()

(187, 3)


,Destino,Link,Referencias
0,Brasil,https://rbbv.com.br/destino/america-do-sul/brasil/,27
1,Acre,https://rbbv.com.br/destino/america-do-sul/brasil/acre/,1
2,Alagoas,https://rbbv.com.br/destino/america-do-sul/brasil/alagoas/,1
3,Amazonas,https://rbbv.com.br/destino/america-do-sul/brasil/amazonas/,1
4,Amapá,https://rbbv.com.br/destino/america-do-sul/brasil/amapa/,1


In [72]:
def count_references_all(lis):
    all_links = [li[1] for li in lis if li[1]]
    return [count_references(li, all_links) for li in all_links]

def count_references(li, lis):
    sub = [l for l in lis if l.startswith(li)]
    return len(sub)

In [41]:
pd.options.display.max_colwidth = 100

In [60]:
all_links = df_destinos.Link.values

In [62]:
brasil = 'https://rbbv.com.br/destino/america-do-sul/brasil/'
acre = 'https://rbbv.com.br/destino/america-do-sul/brasil/acre/'

print(count_references(brasil, all_links))
print(count_references(acre, all_links))

27
1
